# Laboratorium 2: ewaluacja metod uczenia maszynowego

W ramach laboratorium omówiona zostanie ewaluacja metod uczenia maszynowego ze szczególnym uwzględnieniem problemu regresji.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

boston = datasets.load_boston()
print(boston.DESCR)

print("boston (shape): ", boston.data.shape)

print(boston.target)

# Wycinamy kawałek danych do testów:

X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, test_size=0.2, random_state=421, shuffle=True)

print(X_train.shape)

## Regresja liniowa

Przejrzyj poniższy kod.
Policz współczynniki $R^2$ na zbiorze testowym i treningowym.
Jak się je interpretuje? Który powinien być większy?
Policz też błędy średniokwadratowe.

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

def evaluate_reg(reg):
  pred_test = reg.predict(X_test)

  fig, ax = plt.subplots()
  ax.scatter(y_test, pred_test, edgecolors=(0, 0, 0))
  ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
  ax.set_xlabel('Measured')
  ax.set_ylabel('Predicted')
  plt.show()

  print("MSE train: ", mean_squared_error(y_train, reg.predict(X_train))) # TODO
  print("MSE test:  ", mean_squared_error(y_test, pred_test)) # TODO

evaluate_reg(lr)

print("R2 train: ", lr.score(X_train, y_train))
print("R2 test:  ", lr.score(X_test, y_test)) # TODO


## Ridge regression

Przeprowadź analogiczną procedurę jak w poprzednim punkcie, ale z wykorzystaniem *Ridge regression*.
Sprawdź, które współczynniki regresji są równe zero. Co to oznacza? Jakim parametrom w danych odpowiadają te cechy?

In [ ]:
from sklearn import linear_model

reg_ridge = linear_model.Ridge(alpha=1)
reg_lasso = linear_model.Lasso(alpha=0.5)

reg = reg_lasso

reg.fit(X_train, y_train)
evaluate_reg(reg)
print("Params: ", reg.coef_) # TODO
print("Intercept: ", reg.intercept_) # TODO

## solution 
print("R2 train: ", reg.score(X_train, y_train))
print("R2 test:  ", reg.score(X_test, y_test)) # TODO

## Drzewa regresji

Dopasuj kilkukrotnie model drzewa regresji. W jakim zakresie zmienia się błąd na zbiorze treningowym?

In [ ]:
from sklearn import tree

reg = tree.DecisionTreeRegressor()

reg.fit(X_train, y_train)

evaluate_reg(reg)

## Walidacja krzyżowa

Zaobserwuj stabilność średniej z MSE przy wykorzystaniu walidacji krzyżowej.
Dopisz sprawdzanie błędów poszczególnych klasyfikatorów na zbiorze testowym (normalnie tego się **nie** robi -- to jest tylko dla celów poglądowych). Zastanów się skąd może brać się różnica.

Porównaj wyniki dla drzew regresji i metody Lasso z wcześniejszego zadania.

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)
mean_train_f = []
mean_validate_f = []
mean_test = []


for train_idx, valid_idx in kf.split(X_train):
  X_train_f, X_validate_f = X_train[train_idx], X_train[valid_idx]
  y_train_f, y_validate_f = y_train[train_idx], y_train[valid_idx]

  # reg = tree.DecisionTreeRegressor()
  reg = linear_model.Lasso(alpha=0.5)
  reg.fit(X_train_f, y_train_f)

  mean_train_f.append(mean_squared_error(y_train_f, reg.predict(X_train_f)))
  mean_validate_f.append(mean_squared_error(y_validate_f, reg.predict(X_validate_f)))
  mean_test.append(mean_squared_error(y_test, reg.predict(X_test)))

print(mean(mean_train_f))
print(mean(mean_validate_f))
print(mean(mean_test))

# wypisz średnie wartości MSE na zbiorach:
# X_t_f (trening), X_v_f (walidacja), X_test (testowy)

## Optymalizacja parametrów

Z użyciem walidacji krzyżowej

Znajdź optymalną wartość parametru `alpha` dla metody Lasso.

Następnie znajdź optymalne parametry dla metody `DecisionTreeRegressor` (na przykład `max_depth`, `min_samples_leaf`). Spróbuj zmodyfikować także inne parametry i sprawdź rezultaty.

Napisz własną funkcję oceniającą model regresji tak, aby premiowała małe drzewa. Ile minimum węzłów w drzewie potrzebujesz aby MSE na zbiorze testowym było mniejsze niż 15?

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'alpha': [0.1, 0.5 , 1.0, 5.0, 10.0]}
lasso_reg = linear_model.Lasso()
gcv_reg = GridSearchCV(lasso_reg, parameters, scoring='neg_mean_squared_error')
gcv_reg.fit(X_train, y_train)
print(gcv_reg.best_params_)

print("MSE train: ", mean_squared_error(y_train, gcv_reg.predict(X_train))) # TODO
print("MSE test:  ", mean_squared_error(y_test, gcv_reg.predict(X_test))) # TODO

grid_tree = {'max_depth': [10, 50, 100, 500, 1000], 'min_samples_leaf':[1, 2, 3, 4, 5, 10]} # TODO
tree_reg = tree.DecisionTreeRegressor()
gcv_reg = GridSearchCV(tree_reg, grid_tree, scoring='neg_mean_squared_error') # TODO
gcv_reg.fit(X_train, y_train)
print(gcv_reg.best_params_)

tree.plot_tree(gcv_reg.best_estimator_)

print("MSE train: ", mean_squared_error(y_train, gcv_reg.predict(X_train))) # TODO
print("MSE test:  ", mean_squared_error(y_test, gcv_reg.predict(X_test))) # TODO
print("Number of nodes: ", gcv_reg.best_estimator_.tree_.node_count)